##Info
<!-- 

To run this notebook, click menu Cell -> Run All

Workflow:
    1. Sum:
        WW
        GWI
        BSF
        Runoff
        dfs0 inflow
    2. Sum:
        WWTP flow
        MH Spilling
        Outfalls
        Delta volume

 -->

In [ ]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *
import subprocess
import sqlite3
from datetime import datetime as dt, timedelta

In [ ]:
# sql function
def sql_to_df(sql,model):
  con = sqlite3.connect(model)
  df = pd.read_sql(sql, con)
  con.close()
  return df

In [ ]:

flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
cover_types = ['Normal','Spilling']
boundary_inflows = []
boundary_inflows.append('Landfill')

for m in master_list:
  
    model_area = m[0]
    model = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    element_filter = m[7]
    wwtp_pipe = m[9]
    outfall_summary = m[10] 
    
    #find subfolders for MIKE+
    if model[-7:] == '.sqlite':
        result_dict = {}
        not_founds = []
        for r in result_list:
            file = r[1]
            file_found = False
            for f1 in os.listdir(result_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder
                    result_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                    try:
                        for f2 in os.listdir(result_folder + '\\' + result_subfolder):
                            if os.path.basename(f2) == file:
                                result_dict[file] = [f1,'\\' + result_subfolder]
                                file_found = True
                    except:
                        pass
            if not file_found:
                not_founds.append(file)
    
    for r in result_list[:1]:
        header = r[0]
        file = r[1]
        model_path = result_folder + '\\' + result_dict[file][0]
        result_network_path = result_folder + '\\' + result_dict[file][1] + '\\' + file
        result_runoff_path = result_network_path[:-16] + 'Surface_runoff.res1d'

        
        print('Import DWF')
        sql = "SELECT ms_DPProfileD.ScheduleID AS Day_Type, ms_DPPatternD.Sqn AS [Hour], Sum(msm_Loadpoint.loadflow*ms_DPPatternD.DPValue) AS DWF "
        sql += "FROM ((msm_Loadpoint INNER JOIN msm_BBoundary ON msm_Loadpoint.LoadCategoryNo = msm_BBoundary.LoadCategoryNo) INNER JOIN ms_DPProfileD ON msm_BBoundary.DPProfileID = ms_DPProfileD.ProfileID) INNER JOIN ms_DPPatternD ON ms_DPProfileD.PatternID = ms_DPPatternD.PatternID "
        sql += "WHERE msm_Loadpoint.Active = 1 AND ms_DPProfileD.Active = 1 AND ms_DPPatternD.Active = 1 AND msm_BBoundary.Active = 1 "
        sql += "GROUP BY ms_DPProfileD.ScheduleID, ms_DPPatternD.Time "
        sql += "HAVING (LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekday' Or LOWER(SUBSTR(ms_DPProfileD.ScheduleID,1,7))='weekend') AND ms_DPPatternD.Sqn <> 0 "
        sql += "ORDER BY scheduleid, time"
        diurnal_wws = sql_to_df(sql,model_path) 
        
        sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE description = 'Baseflow' and Active = 1"
        gwi = sql_to_df(sql,model_path).iloc[0,0]
        
        diurnal_wws.Hour = diurnal_wws.Hour - 1
        diurnal_wws.DWF = diurnal_wws.DWF + gwi
        
        sql = "SELECT COUNT(muid) FROM msm_Loadpoint WHERE description = 'BSF' and Active = 1"
        bsf_count = sql_to_df(sql,model_path).iloc[0,0]
        bsf = 0
        bsf_on = True if bsf_count > 0 else False
        if bsf_on:
            print('Import BSF')
            sql = "SELECT SUM(loadflow) FROM msm_Loadpoint WHERE description = 'BSF' and Active = 1"
            bsf = sql_to_df(sql,model_path).iloc[0,0]
            sql = "SELECT SUM(area) FROM msm_Catchment WHERE Active = 1"
            area = sql_to_df(sql,model_path).iloc[0,0]
            ini_rate = round(bsf * 86400 / area * 10000 * 1000, 0)
            ini_no = ini_rate / 11200 
            
        #Extract results
        print('Import WWTP')
        res1d = Res1D(result_network_path)
        sim_start = res1d.time_index.min()
        start = sim_start + timedelta(days=1)
        end = res1d.time_index.max()
        sim_seconds = (end - sim_start).total_seconds()
        timesteps = len(res1d.time_index)-1
        timestep_seconds = sim_seconds / timesteps
        skip_steps = int(86400 / timestep_seconds)
        
        wwtp_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
        wwtp_df['WWTP'] = list(res1d.query.GetReachEndValues(wwtp_pipe, "Discharge"))[skip_steps:]
        
        print('Import outfalls')
        outfall_df = pd.read_csv(outfall_summary)
        first_round = True
        for index, row in outfall_df.iterrows():
            muid = row['Structure']
            layer = row['Layer']
            outfall = row['Outfall']
            resid = muid
            if layer.lower() != 'msm_link':
                resid = layer[4:] + ':' + muid

            overflow_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
            ts = list(res1d.query.GetReachEndValues(resid, "Discharge"))[skip_steps:]
            overflow_df['MUID'] = muid
            overflow_df['Overflow'] = ts
            if first_round == True:
                overflow_df_all = overflow_df.copy()
            else:
                overflow_df_all = pd.concat([overflow_df_all,overflow_df])                                                        
            first_round = False
    
        if first_round == False:                        
            overflow_df_all = overflow_df_all.groupby(overflow_df_all.index).agg({'Overflow': 'sum'})        
        
        print('Import inflow')
        has_spill = False
        first_round = True
        for node in res1d.data.Nodes:
            muid = node.Id
            for i, flood_type in enumerate(flood_types):
                ts = res1d.query.GetNodeValues(muid,flood_type)
                if ts != None:
                    if max(ts) > 0:
                        spill_df = pd.DataFrame(index=res1d.time_index)
                        spill_df['Node'] = muid
                        spill_df['Spill'] = ts
                        if first_round == True:
                            spill_df_all = spill_df.copy()
                        else:
                            spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                        first_round = False
                        has_spill = True
        
        if first_round == False:                        
            spill_df_all = spill_df_all.groupby(spill_df_all.index).agg({'Spill': 'sum'})
            
        print('Import runoff')
        has_runoff = False
        first_round = True
        if os.path.exists(result_runoff_path):
            res1d = Res1D(result_runoff_path)
            has_runoff = True
            for i, catchment in enumerate(res1d.data.Catchments):
                ts_id = catchment.Id
                if not ' - RDI' in ts_id and not ' - Kinematic wave (B)' in ts_id:
#                     print('Importing catchment ' + str((i+1)/3) + ' of ' + str(len(res1d.data.Catchments)/3) + ': ' + muid)
                    muid = ts_id
                    ts = res1d.query.GetCatchmentValues(muid,'TotalRunOff')
                    runoff_df = pd.DataFrame(index=res1d.time_index)
                    runoff_df['Node'] = muid
                    runoff_df['Runoff'] = ts
                    if first_round == True:
                        runoff_df_all = runoff_df.copy()
                    else:
                        runoff_df_all = pd.concat([runoff_df_all,runoff_df])                                                        
                    first_round = False
                    
                           
            runoff_df_all = runoff_df_all.groupby(runoff_df_all.index).agg({'Runoff': 'sum'})
            
        has_inflow = False
        print('Import inflow')
        sql = "SELECT tsconnection, timeseriesname FROM msm_BBoundary WHERE active = 1 AND typeno = 9 "
        sql += "AND variationno = 3"
        df = sql_to_df(sql,model_path)
        first_round = True
        for index, row in df.iterrows():
            has_inflow = True
            rel_path = row['tsconnection']
            timeseriesname = row['timeseriesname']
            dfs0_path = os.path.abspath(os.path.join(result_folder, rel_path))
            res = mikeio.read(dfs0_path)
            inflow_df = res.to_dataframe()
            for i, col in enumerate(inflow_df.columns):
                if col == timeseriesname:
                    col_no = i
            inflow_df = inflow_df[[timeseriesname]]
            inflow_df['Boundary'] = timeseriesname
            inflow_df.rename(columns={timeseriesname:'Inflow'},inplace=True)
            inflow_df = inflow_df[['Boundary','Inflow']]
            
            if '(liter per sec)' in str(res.items[col_no]):
                inflow_df.Inflow = inflow_df.Inflow/1000
                
            if first_round == True:
                inflow_df_all = inflow_df.copy()
            else:
                inflow_df_all = pd.concat([inflow_df_all,inflow_df])                                                        
            first_round = False
        
        if first_round == False:    
            inflow_df_all = inflow_df_all.groupby(inflow_df_all.index).agg({'Inflow': 'sum'})   
            
        print('Done')
                
                

        
        
                
                





In [ ]:
bsf = 0

In [ ]:
df_all = wwtp_df.copy()
df_all = pd.merge(df_all, overflow_df_all, left_index=True, right_index=True, how='left')
df_all = pd.merge(df_all, spill_df_all, left_index=True, right_index=True, how='left')
df_all = pd.merge(df_all, inflow_df_all, left_index=True, right_index=True, how='left')
df_all['Total Outflow'] = df_all.WWTP + df_all.Spill + df_all.Overflow
df_all = pd.merge(df_all, runoff_df_all, left_index=True, right_index=True, how='left')
df_all['DateTime'] = df_all.index
df_all['Hour'] = df_all.DateTime.dt.hour
df_all['Weekday'] = df_all['DateTime'].dt.day_name()
df_all['Day_Type'] = 'Weekdays'
df_all.loc[df_all['Weekday']=='Saturday','Day_Type']='Weekends'
df_all.loc[df_all['Weekday']=='Sunday','Day_Type']='Weekends'
df_all = pd.merge(df_all,diurnal_wws[['Day_Type', 'Hour','DWF']],on=['Day_Type', 'Hour'],how='inner')
df_all.set_index('DateTime',inplace=True)
df_all.sort_index(inplace=True)
df_all['DWF'] = df_all['DWF'].rolling('1h').mean()
df_all.fillna(method='bfill',inplace=True)
df_all.drop(columns=['Hour','Weekday','Day_Type'],inplace=True)
df_all['BSF'] = bsf
df_all['Total Inflow'] = df_all.Inflow + df_all.Runoff + df_all.DWF + df_all.BSF
df_all


fig = go.Figure()
for col in df_all.columns:   
    

    fig.add_trace(go.Scatter(x=df_all.index, 
                                     y = df_all[col], 
                                     mode='lines',name=col))

    fig.update_layout(title = 'Model inflows and outflows')
fig.show()



In [ ]:
df_all.describe()

In [ ]:
df_all[['Total Inflow','Total Outflow']].plot()

In [ ]:

#         sql = "SELECT tsconnection, timeseriesname FROM msm_BBoundary WHERE active = 1 AND typeno = 9 "
#         sql += "AND variationno = 3"
#         df = sql_to_df(sql,model_path)
#         for index, row in df.iterrows():
#             rel_path = row['tsconnection']
#             timeseriesname = row['timeseriesname']
#             dfs0_path = os.path.abspath(os.path.join(result_folder, rel_path))
#             res = mikeio.read(dfs0_path)
#             inflow = res.to_dataframe()
#             for i, col in enumerate(inflow.columns):
#                 if col == timeseriesname:
#                     col_no = i
#             inflow = inflow[[timeseriesname]]
#             inflow['Boundary'] = timeseriesname
#             inflow.rename(columns={timeseriesname:'Inflow'},inplace=True)
#             inflow = inflow[['Boundary','Inflow']]
            
#             if '(liter per sec)' in str(res.items[col_no]):
#                 inflow.Inflow = inflow.Inflow/1000
#         inflow
            
            
            

In [ ]:
# # res1d = Res1D(result_network_path)
# skip_steps = 288
# outfall_df = pd.read_csv(outfall_summary)
# first_round = True
# for index, row in outfall_df.iterrows():
#     muid = row['Structure']
#     layer = row['Layer']
#     outfall = row['Outfall']
#     resid = muid
#     if layer.lower() != 'msm_link':
#         resid = layer[4:] + ':' + muid
#     print(resid)
    
#     overflow_df = pd.DataFrame(index=res1d.time_index)[skip_steps:]
#     ts = list(res1d.query.GetReachEndValues(resid, "Discharge"))[skip_steps:]
#     overflow_df['MUID'] = muid
#     overflow_df['Overflow'] = ts
#     if first_round == True:
#         overflow_df_all = overflow_df.copy()
#     else:
#         overflow_df_all = pd.concat([overflow_df_all,overflow_df])                                                        
#     first_round = False
    
# if first_round == False:                        
#     overflow_df_all = overflow_df_all.groupby(overflow_df_all.index).agg({'Overflow': 'sum'})

In [ ]:
overflow_df_all.plot()

In [ ]:
spill_df_all.plot()

In [ ]:
runoff_df_all.plot()